In [8]:
import sys

sys.setrecursionlimit(100000)

In [9]:
import random

In [10]:
class Player:
    def __init__(self, name, num_of_cards):
        """
        The base player class of the game
        Inputs
        -----------
        name = (str) player's name
        num_of_cards = (int) number of cards in the deck
        """
        self.name = name
        self.deck_count = num_of_cards
        self.target = self.deck_count * 2 - 1 #21
        self.cards = []
        self.erases_remaining = self.deck_count // 5 #4
        self.has_stopped = False

    def draw_card(self, card):
        """
        draws a card, and adds it to player cards
        Input
        -------------
        card: (int) the card to be added
        """
        self.cards.append(card)

    def print_info(self):
        """
        prints info of the player
        """
        print(f"{self.name}'s cards: ", end='')
        for c in self.cards:
            print(f'{c}, ', end='')
        print(f'sum: {sum(self.cards)}')
    
    def get_margin(self):
        """
        returns the margin left to target by the player
        Output
        ----------
        (int) margin to target
        """
        return self.target - sum(self.cards) 
    
    def cpu_play(self, seen_cards, deck, enemies_cards):
        """
        The function for cpu to play the game
        Inputs
        ----------
        seen_cards:     (list of ints) the cards that have been seen until now
        deck:           (list of ints) the remaining playing deck of the game
        enemies_cards:  (list of ints) the cards that the enemy currently has.
        Output
        ----------
        (str) a command given to the game
        
        """
        if (len(deck) > 0):
            next_card_in_deck = deck[0]
        else:
            next_card_in_deck = 0
        if (len(deck) > 1):
            next_enemy_card_in_deck = deck[1]
        else:
            next_enemy_card_in_deck = 0
        amount_to_target = self.target - sum(self.cards)
        amount_with_next_card = self.target - (sum(self.cards) + next_card_in_deck)
        enemies_amount_to_target = self.target - sum(enemies_cards)
        enemies_amount_with_next_card = self.target - (sum(enemies_cards) + next_enemy_card_in_deck)
        _stop_condition = amount_to_target < next_card_in_deck and self.erases_remaining <= 0
        _draw_condition_1 = next_card_in_deck != 0
        _draw_condition_2 = amount_with_next_card >= 0
        _erase_condition = self.erases_remaining > 0
        _erase_self_condition = amount_to_target < 0
        _erase_opponent_condition_or = enemies_amount_to_target < (self.target // 7)
        _erase_opponent_condition_or_2 = enemies_amount_with_next_card < (self.target // 7) 
        _erase_opponent_condition_or_3 = enemies_amount_with_next_card <= amount_with_next_card
        _erase_opponent_condition_or_4 = enemies_amount_to_target <= amount_to_target
        _erase_opponent_condition = _erase_opponent_condition_or or _erase_opponent_condition_or_2 or _erase_opponent_condition_or_3
        _erase_opponent_condition = _erase_opponent_condition or _erase_opponent_condition_or_4 
        if (_stop_condition):
            return 'stop'
        elif (_draw_condition_1 and _draw_condition_2):
            return 'draw'
        elif(_erase_self_condition and _erase_condition):
            return 'erase_self'
        elif(_erase_opponent_condition and _erase_condition):
            return 'erase_opponent'
        else:
            return 'stop'
    
    def erase(self, target):
        """
        erases the last card of the target
        Input
        ---------
        target: (Player obj) the player whos last card is about to be erased
        """
        if (len(target.cards) == 0):
            print(f'{target.name} has no more eraseble cards!')
            return
        if (self.erases_remaining > 0):
            self.erases_remaining -= 1
            card = target.cards.pop(-1)
            print(f'{self.name} erased {card} from {target.name}\'s deck!')
            return
        print(f'{self.name} has no more erases remaining!')

    def get_player_cards(self):
        return self.cards

    def get_erases_remained(self):
        return self.erases_remaining

In [71]:
a = ( (10, 11), (6, 2), (15, 16, 17), 5, 1, )
#( (player_cards), (opponent_cards), (deck), player_erase_remaining, opponent_erase_remaining )

d = dict()
d[a] = 5

d


{((10, 11), (6, 2), (15, 16, 17), 5, 1): 5}

In [72]:
a = ['d' for i in range(21)]
for i in range(1, 22):
    if (i % 2):
        a[i - 1] = 'p'
    else:
        a[i - 1] = 'd'
        
str(a)

"['p', 'd', 'p', 'd', 'p', 'd', 'p', 'd', 'p', 'd', 'p', 'd', 'p', 'd', 'p', 'd', 'p', 'd', 'p', 'd', 'p']"

In [11]:
from copy import copy, deepcopy
move = dict()   
        

class Blacksin:
    def __init__(self, deck_count=21):
        """
        The main game class
        Inputs
        -----------
        deck_count = (int) number of cards in the deck
        """
        self.deck_count = deck_count
        self.target = self.deck_count * 2 - 1
        self.player = Player('player', deck_count)
        self.opponent = Player('opponent', deck_count)
        self.deck = self.shuffle_cards()
        self.seen_cards = []
        
    def make_copy(self):
        ret = Blacksin(self.deck_count)
        ret.target = self.target
        
        ret.player.name = self.player.name
        ret.player.deck_count = self.player.deck_count
        ret.player.target = self.player.target
        ret.player.cards = []
        for c in self.player.cards:
            ret.player.cards.append(c)
        ret.player.erases_remaining = self.player.erases_remaining
        ret.player.has_stopped = self.player.has_stopped
        
        
        ret.opponent.name = self.opponent.name
        ret.opponent.deck_count = self.opponent.deck_count
        ret.opponent.target = self.opponent.target
        ret.opponent.cards = []
        for c in self.opponent.cards:
            ret.opponent.cards.append(c)
        ret.opponent.erases_remaining = self.opponent.erases_remaining
        ret.opponent.has_stopped = self.opponent.has_stopped
        
        ret.deck.clear()
        for c in self.deck:
            ret.deck.append(c)
            
        ret.seen_cards = []
        for c in self.seen_cards:
            ret.seen_cards.append(c)
            
        return ret
        
   
        
        #( (player_cards), (opponent_cards), (deck), player_erase_remaining, opponent_erase_remaining )

    def get_state(self):
        ans = ( tuple(self.player.cards), tuple(self.opponent.cards), tuple(self.deck), self.player.erases_remaining, self.opponent.erases_remaining )
        return ans
        
    
    def shuffle_cards(self):
        """ 
        shuffles cards for deck creation
        """
        return list(random.sample(range(1, self.deck_count + 1), self.deck_count))

    def draw_card(self):
        """ 
        draws a card from deck, if non is remaining, ends the game.
        """
        if (len(self.deck) > 0):
            card = self.deck.pop(0)
            self.seen_cards.append(card)
            return card
        print('The deck is empty! ending game...')
        self.opponent.has_stopped = True
        self.player.has_stopped = True
        return -1

    def handout_cards(self):
        """ 
        hands out cards to players
        """
        self.player.draw_card(self.draw_card())
        self.opponent.draw_card(self.draw_card())
        self.player.draw_card(self.draw_card())
        self.opponent.draw_card(self.draw_card())
    
    def handle_input(self, _input, player):
        """ 
        handles input
        Input
        ------------
        _input: (str) input given by the player
        player: (Player obj)the player that is giving the input
        
        """
        if (player is self.player):
            opponent = self.opponent
        else:
            opponent = self.player
        if (_input == 'stop' or _input == 's'):
            player.has_stopped = True
            print(f'{player.name} has stopped')
        elif (_input == 'draw' or _input == 'd'):
            card = self.draw_card()
            if (card == -1): return False
            player.draw_card(card)
            print(f'{player.name} drawed a card: {card}')
        elif ((_input == 'erase_self' or _input == 'es')):
            player.erase(player)
        elif ((_input == 'erase_opponent' or _input == 'eo')):
            player.erase(opponent)
        else:
            print('ERROR: unknown command')
            return False
        return True
    

    def get_player_input(self):
        state = self.get_state()
        your_input = move[state]
        self.handle_input(your_input, self.player)
            
    def opponent_play(self):
        """
        function for opponent to play it's turn
        """
        try:
            opponent_input = self.opponent.cpu_play(self.seen_cards, self.deck, self.player.cards)
        except:
            opponent_input = 'stop'
        self.handle_input(opponent_input, self.opponent)

    def check_for_winners(self):
        """
        checks for winners.
        Output
        -----------
        (int) returns 1 if player wins, 0 if draw and -1 if opponent wins
        """
        self.opponent.print_info()
        self.player.print_info()
        player_margin = self.player.get_margin()
        opponent_margin = self.opponent.get_margin()
        player_win_condition_1 = opponent_margin < 0 and player_margin >= 0
        player_win_condition_2 = opponent_margin >=0 and player_margin >= 0 and player_margin < opponent_margin
        draw_condition_1 = opponent_margin < 0 and player_margin < 0
        draw_condition_2 = opponent_margin >= 0 and player_margin >= 0 and player_margin == opponent_margin
        opponent_win_condition_1 = player_margin < 0 and opponent_margin >= 0
        opponent_win_condition_2 = opponent_margin >=0 and player_margin >= 0 and player_margin > opponent_margin
        if (player_win_condition_1 or player_win_condition_2):
            print(f'the winner is the {self.player.name}!')
            return 1
        elif(draw_condition_1 or draw_condition_2):
            print('the game ends in a draw!')
            return 0
        elif(opponent_win_condition_1 or opponent_win_condition_2):
            print(f'the winner is the {self.opponent.name}!')
            return -1
        else:
            print('an error has accurred! exiting...')
            exit()

    def print_deck(self):
        """
        prints the current deck of the game
        """
        print('full deck: [top] ', end='')
        for i in self.deck:
            print(i, end=' ')
        print('[bottom]')

    def get_winner(self):
        """
        checks for winners.
        Output
        -----------
        (int) returns 1 if player wins, 0 if draw and -1 if opponent wins
        """
        self.opponent.print_info()
        self.player.print_info()
        player_margin = self.player.get_margin()
        opponent_margin = self.opponent.get_margin()
        player_win_condition_1 = opponent_margin < 0 and player_margin >= 0
        player_win_condition_2 = opponent_margin >=0 and player_margin >= 0 and player_margin < opponent_margin
        draw_condition_1 = opponent_margin < 0 and player_margin < 0
        draw_condition_2 = opponent_margin >= 0 and player_margin >= 0 and player_margin == opponent_margin
        opponent_win_condition_1 = player_margin < 0 and opponent_margin >= 0
        opponent_win_condition_2 = opponent_margin >=0 and player_margin >= 0 and player_margin > opponent_margin
        if (player_win_condition_1 or player_win_condition_2):
            #print(f'the winner is the {self.player.name}!')
            return 1
        elif(draw_condition_1 or draw_condition_2):
            #print('the game ends in a draw!')
            return 0
        elif(opponent_win_condition_1 or opponent_win_condition_2):
            #print(f'the winner is the {self.opponent.name}!')
            return -1
        else:
            #print('an error has accurred! exiting...')
            #exit()
            return -2 #have no winner
        
        
    def run(self):
        """
        main function to run the game with
        """
        print('\nstarting game... shuffling... handing out cards...')
        print(f'remember, you are aiming for nearest to: {self.target}')
        self.print_deck()
        self.handout_cards()
        
        cself = self.make_copy()
        
        cself.pre_run(1)
        
        return 1
        
        print("move is")
        print(move)
        
        turn = 0
        while(not self.player.has_stopped or not self.opponent.has_stopped):
            if (turn == 0):
                if (not self.player.has_stopped):
                    self.opponent.print_info()
                    self.player.print_info()
                    self.get_player_input()
                    print()
                    print()
            else:
                if (not self.opponent.has_stopped):
                    print('opponent playing...')
                    self.opponent_play()
                    print()
                    print()
                    
            
            turn = 1 - turn
        print('\nand the winner is...')
        return self.check_for_winners()
    
    
    
    def pre_run(self, is_max):
        print(self.get_state())
        print()
        if ((self.player.has_stopped and self.opponent.has_stopped) or (len(self.deck) == 0)): #terminal state
            return self.get_winner()
        
        if (sum(self.player.cards) > self.target):
            return self.get_winner()
        
        if (sum(self.opponent.cards) > self.target):
            return self.get_winner()
        
        cur_state = self.get_state()
        
        s_util = -1
        d_util = -1
        es_util = -1
        eo_util = -1
        
        stop_util = 1
        draw_util = 1
        erase_self_util = 1
        erase_opponent_util = 1
        
        if is_max:
            s_self = self.make_copy()
            if (s_self.handle_input('s', s_self.player)): #toonesti stop koni
                s_util = s_self.pre_run(0)
            
            #if (s_util == 1)   #pruning
            #      return 1
            
            d_self = self.make_copy()
            if (d_self.handle_input('d', d_self.player)):
                d_util = d_self.pre_run(0)
            
            es_self = self.make_copy()
            if (es_self.handle_input('es', es_self.player)):
                es_util = es_self.pre_run(0)
            
            eo_self = self.make_copy()
            
            if (eo_self.handle_input('eo', eo_self.player)):
                eo_util = eo_self.pre_run(0)
            
            can_win = 0
            can_draw = 0
            if (s_util == 1):
                move[cur_state] = 's'
                can_win = 1
                
            if (d_util == 1):
                move[cur_state] = 'd'
                can_win = 1
                
            if (es_util == 1):
                move[cur_state] = 'es'
                can_win = 1
                
            if (eo_util == 1):
                move[cur_state] = 'eo'
                can_win = 1
                
            if (can_win):
                return 1
                
            if (s_util == 0):
                move[cur_state] = 's'
                can_draw = 1
                
            if (d_util == 0):
                move[cur_state] = 'd'
                can_draw = 1
                
            if (es_util == 0):
                move[cur_state] = 'es'
                can_draw = 1
                
            if (eo_util == 0):
                move[cur_state] = 'eo'
                can_draw = 1
                
            if (can_draw):
                return 0
            
            move[cur_state] = 's'
            return -1 #loses in all cases
                
            
        else:
            stop_self = self.make_copy()
            if (stop_self.handle_input('stop', stop_self.opponent)): #toonesti stop koni
                stop_util = stop_self.pre_run(1)
            
    
            
            draw_self = self.make_copy()
            if (draw_self.handle_input('draw', draw_self.opponent)):
                draw_util = draw_self.pre_run(0)
            
            erase_self_self = self.make_copy()
            if (erase_self_self.handle_input('erase_self', erase_self_self.opponent)):
                erase_self_util = erase_self_self.pre_run(0)
            
            erase_opponent_self = self.make_copy()
            if (erase_opponent_self.handle_input('erase_opponent', erase_opponent_self.opponent)):
                erase_opponent_util = erase_opponent_self.pre_run(0)
            
            can_win = 0
            can_draw = 0
            if (stop_util == -1):
                move[cur_state] = 'stop'
                can_win = 1
                
            if (draw_util == -1):
                move[cur_state] = 'draw'
                can_win = 1
                
            if (erase_self_util == -1):
                move[cur_state] = 'erase_self'
                can_win = 1
                
            if (erase_opponent_util == -1):
                move[cur_state] = 'erase_opponent'
                can_win = 1
                
            if (can_win):
                return -1
                
            if (stop_util == 0):
                #move[cur_state] = 'stop'
                can_draw = 1
                
            if (draw_util == 0):
                #move[cur_state] = 'draw'
                can_draw = 1
                
            if (erase_self_util == 0):
                #move[cur_state] = 'erase_self'
                can_draw = 1
                
            if (erase_opponent_util == 0):
                #move[cur_state] = 'erase_opponent'
                can_draw = 1
                
            if (can_draw):
                return 0
            
            #move[cur_state] = 'stop'
            return 1 #loses in all cases

#game = Blacksin(deck_count=21)


#result = game.run()

In [12]:
A = Blacksin()

B = A.make_copy()

print(A.player.cards)
print(B.player.cards)

A.player.cards.append(10)
print(A.player.cards)
print(B.player.cards)



[]
[]
[10]
[]


In [ ]:
import copy

class A:
    def __init__(self):
        self.a = 2
        self.b = 5
        
    def run(self):
        self.a += 1
        
    def make_copy(self):
        ret = A()
        
        
a = A()
b = copy.deepcopy(a)
print(a.a, a.b)
print(b.a, b.b)

a.run()

print(a.a, a.b)
print(b.a, b.b)

#c = (1, 2, 3)
#e = (4, 2)
#d = dict()
#d[c + e] = 1
#print(d)

In [ ]:
# Example of using the game class
game = Blacksin(deck_count=21)

game.pre_run(0)


result = game.run()